In [20]:
import pandas as pd
import random
import time
import pymongo
from pymongo import MongoClient
from parsel import Selector
from datetime import datetime
from time import sleep
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup

PROXY = "" # HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={PROXY}')

driver = uc.Chrome(executable_path=r'chromedriver', options=chrome_options)
wait = WebDriverWait(driver, 10)

connection = MongoClient("mongodb://localhost:27017")
db = connection.IndeedJobPostDB
df = pd.DataFrame(list(db.Job_Posts.find()))
df

,_id,Title,Job_Type,Company,Salary,Schedule,Location,Date_Posted,Current_Date,Summary,Apply_link
0,647483c0b4ccc7f531eab330,Host,,Chili's,Estimated $22.9K - $29K a year,,"Little Falls, NJ 07424",Just posted,23-05-29,Flexible part-time or full-time schedule. Comm...,https://www.indeed.com//rc/clk?jk=1f9dd7702acd...
1,647483c0b4ccc7f531eab331,Server,,Chili's,Estimated $19.2K - $24.3K a year,,"Staten Island, NY 10314 (Richmondtown area)",Just posted,23-05-29,Flexible part-time or full-time schedule. Prov...,https://www.indeed.com//rc/clk?jk=702b98a418b1...
2,647483c0b4ccc7f531eab332,Line Cook,,Chili's,$15.00 - $20.25 an hour,,"Glendale, NY 11385",Just posted,23-05-29,Keep food orders flowing continuously from the...,https://www.indeed.com//rc/clk?jk=c4ef85c7a19a...
3,647483c0b4ccc7f531eab333,Restaurant Busser,,AOI Japanese Restaurant,From $14.50 an hour,,"Iselin, NJ 08830",Just posted,23-05-29,"As a busser, you will be responsible for clean...",https://www.indeed.com//rc/clk?jk=7b3c4c12a72d...
4,647483c0b4ccc7f531eab334,Assistant Food and Beverage Outlets Manager,,APA Hotel Woodbridge,"$45,000 - $55,000 a year",,"Iselin, NJ 08830",Just posted,23-05-29,Manage inventory ensuring outlets are fully st...,https://www.indeed.com//company/TKP-Bar-&-Gril...
...,...,...,...,...,...,...,...,...,...,...,...
925,647483c0b4ccc7f531eab6cd,Team Member,,"036823,1347 KENNEDY BLVD,,BAYONNE,NJ",Estimated $29.8K - $37.7K a year,,"Bayonne, NJ 07002",Posted 5 days ago,23-05-29,"As a Pizza Hut team member, you can be the smi...",https://www.indeed.com//rc/clk?jk=e5d8f6888673...
926,647483c0b4ccc7f531eab6ce,Line Cook,,DoubleTree by Hilton Newark Penn Station,Estimated $33.1K - $41.9K a year,,"Newark, NJ 07102 (Central Business District area)",Posted 5 days ago,23-05-29,"Attached to Newark Penn Station, The Doubletre...",https://www.indeed.com//rc/clk?jk=efae4767cff2...
927,647483c0b4ccc7f531eab6cf,Team Member,,"036817,180 MAIN ST,,WEST ORANGE,NJ",Estimated $29.8K - $37.7K a year,,"West Orange, NJ 07052",Posted 5 days ago,23-05-29,"As a Pizza Hut team member, you can be the smi...",https://www.indeed.com//rc/clk?jk=c0f14b638ab6...
928,647483c0b4ccc7f531eab6d0,Line Cook,,Park Avenue Club,$20 - $25 an hour,,"Florham Park, NJ",EmployerActive 2 days ago,23-05-29,Immediately informing a manager or supervisor ...,https://www.indeed.com//company/Park-Avenue-Cl...


In [21]:
df['Apply_link'][10]

'https://www.indeed.com//rc/clk?jk=58930a5669ba4c1a&fccid=0561fa893fc345fb&vjs=3'

In [34]:
links = df['Apply_link']
#for link in links:
driver.get(df['Apply_link'][15])
soup = BeautifulSoup(driver.page_source, features="html.parser")
#print(soup.text)
all_metadata = soup.find_all("div", "css-tvvxwd ecydgvn1")
rows =[i.get_text(strip=True) for i  in soup.find_all('div', 'css-tvvxwd ecydgvn1') if i.get_text(strip=True)!=""]
job_type = []
for i in rows:
    if "$" in i:
        salary = i
        rows.remove(i)
for j in rows:
    if "-time" in j:
        job_type.append(j)
        rows.remove(j)
schedule = ", ".join(rows)
print(salary)
print(", ".join(job_type))
print(schedule)

$18 - $22 an hour

Employee discount, Flexible schedule, Opportunities for advancement, Paid sick time, Paid time off


In [32]:
def post_facebook(email, password, post):
    email = email
    password = password
    post = post

    driver.get('https://facebook.com/')

    email_input = driver.find_element(By.XPATH, '//*[@id="email"]')
    email_input.send_keys(email)
    password_input = driver.find_element(By.XPATH, '//*[@id="pass"]')
    password_input.send_keys(password)
    login_button = driver.find_element(By.TAG_NAME, 'button')
    login_button.click()
    wait.until(EC.url_changes('https://facebook.com/'))
    driver.get('https://www.facebook.com/groups/1620236638072327')
    actions= ActionChains(driver)
    actions.send_keys(Keys.ESCAPE).perform()
    try:
        post_box = driver.find_element(By.XPATH, "//*[text()='Write something...']")
    except:
        post_box = driver.find_element(By.XPATH, "//*[@class='x1lliihq x6ikm8r x10wlt62 x1n2onr6' or @style='-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;']")

    post_box.click()
    actions_post = ActionChains(driver)
    actions_post.send_keys(post).perform()

    post_button = driver.find_element(By.XPATH, "//*[text()='Post']")
    post_button.click()

Enter your email: rittiquealam@gmail.com
Enter your password: KillMe@2030
Enter your post: Test 05
